<a href="https://colab.research.google.com/github/Abdul-ur-Rehman-Butt/2022-SE-27_LAB_TASKS_ML/blob/main/lab_10_2022_SE_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score


In [18]:
data  = pd.read_csv('/content/advertising_ef.csv')

In [19]:
data

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,City,Gender,Country,Clicked on Ad
0,68.95,35.0,61833.90,256.09,Wrightburgh,Female,Tunisia,0
1,NaN,31.0,68441.85,193.77,West Jodi,Male,Nauru,0
2,69.47,26.0,59785.94,236.50,Davidton,Female,San Marino,0
3,74.15,29.0,54806.18,245.89,West Terrifurt,Male,Italy,0
4,68.37,35.0,73889.99,225.58,South Manuel,Female,Iceland,0
...,...,...,...,...,...,...,...,...
1004,72.97,30.0,71384.57,208.58,Duffystad,Male,Lebanon,1
1005,51.30,45.0,67782.17,134.42,New Darlene,Male,Bosnia and Herzegovina,1
1006,51.63,51.0,42415.72,120.37,South Jessica,Male,Mongolia,1
1007,55.55,19.0,41920.79,187.95,West Steven,Female,Guatemala,0


In [21]:
# Handling missing values
# Fill numerical columns with median values
data.fillna({
    "Daily Time Spent on Site": data["Daily Time Spent on Site"].median(),
    "Age": data["Age"].median(),
    "Area Income": data["Area Income"].median(),
    "Daily Internet Usage": data["Daily Internet Usage"].median()
}, inplace=True)

# Fill categorical columns with mode (most frequent value)
data["City"].fillna(data["City"].mode()[0], inplace=True)
data["Country"].fillna(data["Country"].mode()[0], inplace=True)

# Outlier detection and removal using IQR method
numerical_columns = ["Daily Time Spent on Site", "Age", "Area Income", "Daily Internet Usage"]

for col in numerical_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove outliers
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

<ipython-input-21-89e3968da990>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["City"].fillna(data["City"].mode()[0], inplace=True)
<ipython-input-21-89e3968da990>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [22]:
data.describe()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad
count,999.000000,999.000000,999.000000,999.000000,999.000000
mean,65.080370,35.926927,55361.966712,180.083684,0.496496
std,15.904141,8.747268,12814.364764,43.987300,0.500238
min,32.600000,19.000000,20592.990000,104.780000,0.000000
25%,51.340000,29.000000,47419.920000,138.790000,0.000000
50%,68.470000,35.000000,56998.245000,183.420000,0.000000
75%,78.590000,42.000000,65254.645000,218.885000,1.000000
max,91.430000,61.000000,79484.800000,269.960000,1.000000


In [23]:
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,City,Gender,Country,Clicked on Ad
0,68.95,35.0,61833.90,256.09,Wrightburgh,Female,Tunisia,0
1,68.37,31.0,68441.85,193.77,West Jodi,Male,Nauru,0
2,69.47,26.0,59785.94,236.50,Davidton,Female,San Marino,0
3,74.15,29.0,54806.18,245.89,West Terrifurt,Male,Italy,0
4,68.37,35.0,73889.99,225.58,South Manuel,Female,Iceland,0


In [24]:
# Splitting into features (X) and target variable (y)
X = data.drop(columns=['Clicked on Ad'])
y = data['Clicked on Ad']


In [27]:
# Encoding categorical features
label_enc = LabelEncoder()
X['City'] = label_enc.fit_transform(X['City']) # Categorical: Convert to numbers
X['Country'] = label_enc.fit_transform(X['Country']) # Categorical: Convert to numbers
X['Gender'] = label_enc.fit_transform(X['Gender']) # Binary: Convert to 0 and 1


In [28]:
# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
# Gaussian Naïve Bayes (for numerical data: Age,Area Income, daily time spent on site)
scaler = StandardScaler()
X_train_gnb = scaler.fit_transform(X_train[['Age', 'Area Income','Daily Time Spent on Site']])
X_test_gnb = scaler.transform(X_test[['Age', 'Area Income','Daily Time Spent on Site']])

gnb = GaussianNB()
gnb.fit(X_train_gnb, y_train)
probs_gnb = gnb.predict_proba(X_test_gnb) # Get probability outputs

In [33]:
# Multinomial Naïve Bayes (for categorical data: city, country)
mnb = MultinomialNB()
mnb.fit(X_train[['City','Country']], y_train)
probs_mnb = mnb.predict_proba(X_test[['City','Country']])


In [34]:
# Bernoulli Naïve Bayes (for binary data: gender)
bnb = BernoulliNB()
bnb.fit(X_train[['Gender']], y_train)
probs_bnb = bnb.predict_proba(X_test[['Gender']])

In [35]:
# **Ensemble Using Probability Multiplication**
ensemble_probs = probs_gnb * probs_mnb * probs_bnb # Multiply probabilities
ensemble_probs = ensemble_probs / ensemble_probs.sum(axis=1, keepdims=True) # Normalize

In [36]:
# Final predictions
final_predictions = np.argmax(ensemble_probs, axis=1)

In [37]:
# Accuracy of the ensemble model
ensemble_accuracy = accuracy_score(y_test, final_predictions)


In [38]:
# Print accuracies
print("Gaussian Naïve Bayes Accuracy:", accuracy_score(y_test, np.argmax(probs_gnb, axis=1)))
print("Multinomial Naïve Bayes Accuracy:", accuracy_score(y_test, np.argmax(probs_mnb, axis=1)))
print("Bernoulli Naïve Bayes Accuracy:", accuracy_score(y_test, np.argmax(probs_bnb, axis=1)))
print("Ensemble Model Accuracy:", ensemble_accuracy)


Gaussian Naïve Bayes Accuracy: 0.9133333333333333
Multinomial Naïve Bayes Accuracy: 0.5
Bernoulli Naïve Bayes Accuracy: 0.5266666666666666
Ensemble Model Accuracy: 0.86


In [39]:
# Convert probabilities to DataFrame for better understanding in the form of table
prob_df = pd.DataFrame({
'Actual Target': y_test.values,
'GNB - P(No)': probs_gnb[:, 0], 'GNB - P(Yes)': probs_gnb[:, 1],
'MNB - P(No)': probs_mnb[:, 0], 'MNB - P(Yes)': probs_mnb[:, 1],
'BNB - P(No)': probs_bnb[:, 0], 'BNB - P(Yes)': probs_bnb[:, 1],
'Ensemble - P(No)': ensemble_probs[:, 0], 'Ensemble - P(Yes)': ensemble_probs[:, 1],
'Final Prediction': final_predictions
})
# Map 0 -> "No", 1 -> "Yes" for better readability
prob_df['Actual Target'] = prob_df['Actual Target'].map({0: "No", 1: "Yes"})
prob_df['Final Prediction'] = prob_df['Final Prediction'].map({0: "No", 1: "Yes"})
prob_df

,Actual Target,GNB - P(No),GNB - P(Yes),MNB - P(No),MNB - P(Yes),BNB - P(No),BNB - P(Yes),Ensemble - P(No),Ensemble - P(Yes),Final Prediction
0,No,0.974664,0.025336,0.730119,0.269881,0.523420,0.476580,9.913272e-01,0.008673,No
1,No,0.975964,0.024036,0.860048,0.139952,0.487563,0.512437,9.958055e-01,0.004194,No
2,Yes,0.000004,0.999996,0.028345,0.971655,0.523420,0.476580,1.130785e-07,1.000000,Yes
3,No,0.989353,0.010647,0.143535,0.856465,0.487563,0.512437,9.367785e-01,0.063222,No
4,Yes,0.238803,0.761197,0.186792,0.813208,0.523420,0.476580,7.333928e-02,0.926661,Yes
...,...,...,...,...,...,...,...,...,...,...
295,No,0.945030,0.054970,0.008311,0.991689,0.523420,0.476580,1.366177e-01,0.863382,Yes
296,No,0.976983,0.023017,0.946063,0.053937,0.487563,0.512437,9.985903e-01,0.001410,No
297,Yes,0.000576,0.999424,0.990440,0.009560,0.523420,0.476580,6.158643e-02,0.938414,Yes
298,Yes,0.000640,0.999360,0.513514,0.486486,0.487563,0.512437,6.431953e-04,0.999357,Yes
